# Prototype Financial Planner

In [74]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
from MCForecastTools import MCSimulation
import json
import re

load_dotenv()

True

## Part 1 - Personal Finance Planner
---
### Collect Crypto Prices Using the **requests** library

In [99]:
# We will use default crypto values of 1.2 BTC and 5.3 ETH
my_btc = 1.2
my_eth = 5.3

# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/"

# Store and parse crypto data from alternative.me free API
btc_response = requests.get(btc_url)
btc_data = btc_response.json()
eth_response = requests.get(eth_url)
eth_data = eth_response.json()

bulk_str = str(eth_data['data'].values())
eth_price_match = re.search(r"(price': )(\d+.\d\d)",bulk_str)
eth_price = float(eth_price_match.group(2))
# eth_data['data'][main_key]

# btc_data['data']['1']['quotes']['USD']['price']    
# eth_data['data']

eth_price


2956.95

In [100]:

def find_crypto_value(num_coins, crypto_json,round_to):
    bulk_str = str(crypto_json['data'].values())
    coin_price_match = re.search(r"(price': )(\d+.\d\d)",bulk_str)
    coin_price = float(coin_price_match.group(2))
    crypto_value = round(coin_price*num_coins,int(round_to))
    return crypto_value


my_btc_value = find_crypto_value(my_btc, btc_data, 2)

my_btc_value



# my_eth_value

# print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
# print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

50689.2

TypeError: Object of type method is not JSON serializable